# Test Notebook

Before run this notebook:
-  Build the project first
```bash
mkdir build
cd build
cmake ..
make test
```
- And move this notebook to the bin folder
```bash
mv test.ipynb ../bin
```

## Code

In [11]:
import random
import subprocess

Read the dictionary

In [12]:
words = []
with open("../data/american-english-sorted", 'r') as f:
    for word in f.readlines():
        words.append(word.replace('\n', ''))

Generate random prefixes

In [13]:
num_samples = 1000

In [14]:
prefixes = []
for word in random.sample(words, num_samples):
    while(len(word) < 2):
        word = random.choice(words)
    rand_int = random.randint(1, len(word)-1)
    prefix = word[:rand_int]
    prefixes.append(prefix)

In [15]:
prefixes[:5]

['L', 'v', 'coron', 'miscr', 'fla']

To generate random prefixes with a fix length:

In [16]:
def get_random(prefix_len: int):
    while True:
        word = random.choice(words)
        if len(word) >= prefix_len:
            break
    return word[:prefix_len]

def get_result(prefix: str):
    return [x for x in words if x.startswith(prefix)]

In [17]:
prefix = get_random(prefix_len=4)
results = get_result(prefix)
print(prefix) 
print(results)

indu
['indubitable', 'indubitably', 'induce', 'induced', 'inducement', "inducement's", 'inducements', 'induces', 'inducing', 'induct', 'inductance', "inductance's", 'inducted', 'inductee', "inductee's", 'inductees', 'inducting', 'induction', "induction's", 'inductions', 'inductive', 'inducts', 'indue', 'indued', 'indues', 'induing', 'indulge', 'indulged', 'indulgence', "indulgence's", 'indulgences', 'indulgent', 'indulgently', 'indulges', 'indulging', 'industrial', 'industrialism', "industrialism's", 'industrialist', "industrialist's", 'industrialists', 'industrialization', "industrialization's", 'industrialize', 'industrialized', 'industrializes', 'industrializing', 'industrially', 'industries', 'industrious', 'industriously', 'industriousness', "industriousness's", 'industry', "industry's"]


In [18]:
def get_cpp_results(text: str):
    program_path = './test_autocomplete'
    arguments = [text]
    subprocess.run([program_path] + arguments, text=True, capture_output=True)
    with open("output.txt", 'r') as f:
        result = [x.replace('\n', '') for x in f.readlines()]
    return result

def get_expected_results(text):
    return [x for x in words if x.startswith(text)]

def check(text):
    return get_cpp_results(text) == get_expected_results(text)

In [19]:
check('lamasae')

True

In [20]:
for prefix in prefixes[:1000]:
    print(f"Checking {prefix}")
    if not check(prefix):
        raise Exception(f"Failed for {prefix}")

Checking L
Checking v
Checking coron
Checking miscr
Checking fla
Checking unflinch
Checking Krist
Checking coro
Checking restricti
Checking Con
Checking anarchis
Checking le
Checking Carrie
Checking be
Checking Me
Checking Vesalius'
Checking Patr
Checking nuisan
Checking N
Checking weapon
Checking milli
Checking am
Checking travest
Checking diagno
Checking Wa
Checking br
Checking Jan
Checking Cl
Checking uninsta
Checking dwelling'
Checking Dell'
Checking te
Checking melodi
Checking resemb
Checking smu
Checking deadb
Checking h
Checking ba
Checking campu
Checking so
Checking Vern
Checking Esth
Checking mastu
Checking archivist
Checking wen'
Checking mark
Checking custod
Checking bridg
Checking cou
Checking airbor
Checking quote
Checking s
Checking part
Checking ecclesiastic
Checking p
Checking t
Checking whiz
Checking c
Checking warh
Checking bask
Checking consternati
Checking al
Checking congratulat
Checking v
Checking cheerine
Checking s
Checking reductio
Checking ringwo
Checking kett